In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.0.1'

# Graph 

일반적으로 그래프는 1개만 이용하면 됩니다. 기본적으로 세션은 생성 시 아무런 인자도 주지 않으면 디폴트 그래프를 실행하게 됩니다. 인자를 주면 해당 그래프의 세션으로 만들어지게 됩니다. 
* ex) sess = tf.Session(graph=g)

굳이 그래프를 여러개 만들고 싶은 경우는 어떻게 해야 할까요?
* 여러개의 그래프를 만들면, 각각의 그래프는 각각의 세션을 필요로 하게 되며, 각각의 세션은 가능한 모든 자원을 이용하려고 시도하게 될 것입니다 (by default)
* 다른 그래프 간에는 데이터의 전달도 불가능합니다 (굳이 python이나 numpy로는 가능)

그럼에도 불구하고 굳이 여러개의 그래프를 만들고 싶다면 가장 좋은 방법은 하나의 그래프 내에 여러개의 서브그래프를 만드는 것을 권장합니다. 

g 라는 그래프를 만들어 보겠습니다. 

In [3]:
g = tf.Graph()

g를 디폴트 그래프로 만들어 주고, g 그래프 내에 x라는 변수를 만들어 줍니다.

In [4]:
with g.as_default():
    x = tf.add(3,5)

g 그래프를 실행하는 세션 sess를 정의해 준 다음에, sess를 통해 x의 값을 구합니다.

In [5]:
sess = tf.Session(graph=g)
sess.run(x)

8

만약 sess를 g 그래프가 아닌 다른 그래프에 대해 정의해 주면, 해당 그래프에는 x가 없기 때문에 에러가 발생하게 됩니다. 

In [11]:
# 에러 발생 ("is not an element of this graph")
# with tf.Session() as sess:
#     sess.run(x)

그래프에 연산자를 추가해 주려면 반드시 그래프를 디폴트로 설정해 주고 진행해야 합니다. 

In [12]:
g = tf.Graph()

with g.as_default():
    a = 3
    b = 5
    x = tf.add(a, b)

sess = tf.Session(graph=g)
sess.run(x)

8

디폴트 그래프를 핸들링하기 위한 핸들러의 생성은 다음과 같이 하면 됩니다. 

In [13]:
g = tf.get_default_graph()

디폴트 그래프와 유저가 생성한 그래프를 mix하는 것은 좋은 방법이 아닙니다.

에러를 유발하기 쉽기 때문입니다. 

유저가 그래프를 생성합니다. 

In [14]:
g = tf.Graph()

ops를 추가합니다. 이 ops는 유저 생성 그래프가 아니라 디폴트 그래프에 추가됩니다. 

In [15]:
a = tf.constant(3)

이번에는 유저 생성 그래프에 ops를 추가합니다.

In [16]:
with g.as_default():
    b = tf.constant(5)

sess_default는 디폴트 세션을 실행하도록 만들었습니다.

In [17]:
sess_default = tf.Session()
sess_default.run(a)
# sess_default.run(b) # error

3

sess_user는 유저 생성 세션을 실행하도록 만들었습니다.

In [18]:
sess_user = tf.Session(graph=g)
sess_user.run(b)
# sess_user.run(a) # error

5

In [19]:
c = tf.constant(10)

In [21]:
sess_default.run(c)
# sess_user.run(c) # error

10

그래프를 만들 때 아래 두가지 방법은 동일하다.

In [22]:
# 1. Using Graph.as_default():
g = tf.Graph()
with g.as_default():
  c = tf.constant(5.0)
  assert c.graph is g # no error

In [23]:
# 2. Constructing and making default:
with tf.Graph().as_default() as g:
  c = tf.constant(5.0)
  assert c.graph is g # no error

디폴트 그래프와 유저가 생성한 그래프를 mix하는 것은 좋은 방법이 아닙니다.

그나마 나은 방법은 아래와 같습니다. 

디폴트 그래프에 대한 핸들러를 생성합니다. 

In [24]:
g1 = tf.get_default_graph()

유저 생성 그래프를 생성합니다.

In [25]:
g2 = tf.Graph()

In [26]:
with g1.as_default(): # 명시적으로 (explicitly) 디폴트 그래프화
    a = tf.constant(3) # 디폴트 그래프에 추가

In [27]:
with g2.as_default(): # g2도 디폴트 그래프화
    b = tf.constant(5)

다시 한번 말하지만, 왠만하면 1개의 그래프로 구현 가능하니까 굳이 여러개의 그래프를 만들지 않는 것이 좋습니다.